In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("dwbfgreyhoundwin24032020.csv")
df = df[["EVENT_ID", "EVENT_NAME", "EVENT_DT", "SELECTION_NAME", "BSP", "WIN_LOSE"]]

df["LENGTH"] = df["EVENT_NAME"].str.split().str[1]
df["LENGTH"] = df["LENGTH"].str.replace("m","").astype(int)

df["SELECTION_NUMBER"] = df["SELECTION_NAME"].str.extract('(\d+)').astype(int)

df = df[df["LENGTH"] <= 450]
df["EVENT_ID"] = df["EVENT_ID"].astype(int)
df = df.sort_values(by=["EVENT_DT", "EVENT_ID", "SELECTION_NUMBER"])

# df.dtypes
df.head()

,EVENT_ID,EVENT_NAME,EVENT_DT,SELECTION_NAME,BSP,WIN_LOSE,LENGTH,SELECTION_NUMBER
107,170213450,R1 390m Mdn,23-03-2020 04:37,1. Ignatious Bale,4.14,1,390,1
737,170213450,R1 390m Mdn,23-03-2020 04:37,3. Windburn,21.20,0,390,3
1516,170213450,R1 390m Mdn,23-03-2020 04:37,4. Silver Arapaho,10.69,0,390,4
1515,170213450,R1 390m Mdn,23-03-2020 04:37,5. Double Zero,3.42,0,390,5
1304,170213450,R1 390m Mdn,23-03-2020 04:37,6. Wishly Bale,11.89,0,390,6


In [96]:
#can use dictionary key and value for selection_number and BSP later on
first_dog = 0
for dog in df.itertuples():
    first_dog = dog
    break

winner_odds = 0
winning_dog = first_dog.SELECTION_NUMBER
race_prices = []
race_prices.append(first_dog.BSP)
prev_event = first_dog.EVENT_ID
if first_dog.WIN_LOSE == 1:
    winner_odds = first_dog.BSP
    winning_dog = first_dog.SELECTION_NUMBER

eligible_races = 0
wins = 0
units = 0
losses = 0
loss_prices = 1
races = 0
skip_race = True
lay_dog_number = 0
lay_dog_bsp = 0

for dog in df.itertuples():
    if dog.Index != first_dog.Index:
        if dog.EVENT_ID == prev_event:
#             prev_event = dog.EVENT_ID
            race_prices.append(dog.BSP)
            
            if dog.BSP == min(race_prices):
                lay_dog = dog.SELECTION_NUMBER
                lay_dog_bsp = dog.BSP
            
            if dog.WIN_LOSE == 1:
                winner_odds = dog.BSP
                winning_dog = dog.SELECTION_NUMBER
        
        #check for profits in race_prices
        #if it reaches the first dog for the next event
        
        #evaluating race
        else:
            if 2 < lay_dog < 7 and lay_dog_bsp < 2.5 and len(race_prices) > 6:
                skip_race = False
                eligible_races += 1
            else:
                skip_race = True
                
            if skip_race == False:
                if lay_dog != winning_dog:
                    units += 1
                    wins += 1
                    loss_prices = 1
                else:
                    loss_prices *= lay_dog_bsp
                    if loss_prices > 8:
                        units -= loss_prices-1
                        loss_prices = 1
                
        if dog.EVENT_ID != prev_event:
            race_prices = [dog.BSP]
            races+=1
        if dog.WIN_LOSE == 1:
                winner_odds = dog.BSP
                winning_dog = dog.SELECTION_NUMBER
        
        prev_event = dog.EVENT_ID
        
print(eligible_races)
print(wins)
print(units)
print(races)

7
2
-6.455388523974392
117
